# Benchmarking and Improving OCR System for Southeast Asian Languages

## Data Collection

This section describes the process of collecting the data from Wikipedia.

In [4]:
from data_collection.article_pdf import ArticlePDF
from data_collection.download import download_article_texts, download_article_pdfs, get_articles_by_language

### Collecting Articles in English

I chose 100 Wikipedia articles as my dataset. These 100 articles are a collection of the top 20 most viewed Wikipedia articles from 5 categories ([Wikipedia:Popular pages](https://en.wikipedia.org/wiki/Wikipedia:Popular_pages)). From this list, I collected the article's URL file path in English.

In [2]:
people = ['Donald_Trump', 'Elizabeth_II', 'Barack_Obama', 'Christiano_Ronaldo', 'Michael_Jackson', 'Elon_Musk', 'Lady_Gaga', 'Adolf_Hitler', 'Eminem', 'Lionel_Messi', 'Justin_Bieber', 'Freddie_Mercury', 'Kim_Kardashian', 'Johnny_Depp', 'Steve_Jobs', 'Dwayne_Johnson', 'Michael_Jordan', 'Taylor_Swift', 'Stephen_Hawking', 'Kanye_West']
countries = ['United_States', 'India', 'United_Kingdom', 'Canada', 'Australia', 'China', 'Russia', 'Japan', 'Germany', 'France', 'Singapore', 'Israel', 'Pakistan', 'Philippines', 'Brazil', 'Italy', 'Netherlands', 'New Zealand', 'Ukraine', 'Spain']
cities = ['New_York_City', 'London', 'Hong_Kong', 'Los_Angeles', 'Dubai', 'Washington,_D.C.', 'Paris', 'Chicago', 'Angelsberg', 'Mumbai', 'San_Francisco', 'Rome', 'Monaco', 'Toronto', 'Tokyo', 'Philadelphia', 'Machu_Picchu', 'Jerusalem', 'Amsterdam', 'Boston'] # Excluding Singapore since listed for countries
life = ['Cat', 'Dog', 'Animal', 'Lion', 'Coronavirus', 'Tiger', 'Human', 'Dinosaur', 'Elephant', 'Virus', 'Horse', 'Photosynthesis', 'Evolution', 'Apple', 'Bird', 'Mammal', 'Potato', 'Polar_bear', 'Shark', 'Snake']
structures = ['Taj_Mahal', 'Burj_Khalifa', 'Statue_of_Liberty', 'Great_Wall_of_China', 'Eiffel_Tower', 'Berlin_Wall', 'Stonehenge', 'Mount_Rushmore', 'Colosseum', 'Auschwitz_concentration_camp', 'Great_Pyramid_of_Giza', 'One_World_Trade_Center', 'Empire_State_Building', 'White_House', 'Petra', 'Large_Hadron_Collider', 'Hagia_Sophia', 'Golden_Gate_Bridge', 'Panama_Canal', 'Angkor_Wat'] # Excluding Machu Picchu since listed for cities

english_titles = people + countries + cities + life + structures

english_articles = []
for english_title in english_titles:
    url = f'https://en.wikipedia.org/wiki/{english_title}'
    article = ArticlePDF(english_title, english_title, url, 'en')
    english_articles.append(article)

### Collecting Articles in SEA Languages

Using the [MediaWiki Action API](https://www.mediawiki.org/wiki/API:Main_page), I then collected the articles' URLs and names in the following languages:
- Thai
- Vietnamese
- Bahasa Indonesian


I referred to the [Wikimedia Language Codes](https://www.wikidata.org/wiki/Help:Wikimedia_language_codes/lists/all) to identify the mapping of languages to language codes used by the API.

The API failed to fetch some English articles in other languages. The missing articles include:
- Thai:
    - Christiano_Ronaldo
    - Angelsberg
- Vietnamese:
    - Christiano_Ronaldo
    - Angelsberg
- Bahasa:
    - Christiano_Ronaldo
    - Angelsberg

In [3]:
thai_articles = get_articles_by_language(english_articles, 'th')
vietnamese_articles = get_articles_by_language(english_articles, 'vi')
bahasa_articles = get_articles_by_language(english_articles, 'id')

Issue fetching for Christiano_Ronaldo in th
Issue fetching for Angelsberg in th
Issue fetching for Christiano_Ronaldo in vi
Issue fetching for Angelsberg in vi
Issue fetching for Christiano_Ronaldo in id
Issue fetching for Angelsberg in id


### Downloading Article PDFs and Text

At this point, I had collected the following meta data for each article in each language:
- Article title
- Article title in English
- Article URL
- Language of article

Using [Selenium](https://selenium-python.readthedocs.io/), I then downloaded the articles in PDF format. These PDFs serve as my dataset of images to perform OCR on.

Lastly, using [MediaWiki Action API](https://www.mediawiki.org/wiki/API:Main_page), I downloaded the text of the articles into `.txt` files. These files serve as my ground truth.

Some things to note:
- Downloading the articles take around 10-15 minutes.
- You can change the destination folder to store the downloaded files by modifying the arguments passed into `download_article_pdfs` and `download_article_texts`. Currently, it's set to `data/<language>`.

In [4]:
download_article_pdfs(english_articles, '../data/english')
download_article_pdfs(thai_articles, '../data/thai')
download_article_pdfs(vietnamese_articles, '../data/vietnamese')
download_article_pdfs(bahasa_articles, '../data/bahasa')

download_article_texts(english_articles, '../data/english')
download_article_texts(thai_articles, '../data/thai')
download_article_texts(vietnamese_articles, '../data/vietnamese')
download_article_texts(bahasa_articles, '../data/bahasa')

## Data Preprocessing

This section describes the steps done to pre-process the collected data.

### Converting PDFs to Images

As most OCR tools do not read documents in PDF format, I converted the collected PDFs into PNG images. If the PDF file has multiple pages, then multiple PNG images are stored to represent each page.

In [2]:
from preprocess import convert_pdfs_to_pngs

convert_pdfs_to_pngs('../data/english')
convert_pdfs_to_pngs('../data/thai')
convert_pdfs_to_pngs('../data/vietnamese')
convert_pdfs_to_pngs('../data/bahasa')

## Benchmarking

This section describes the steps done to benchmark OCR tools across different languages using the collected data.

### Iteration v1

In this iteration, I benchmarked on the 20 shortest English articles to get preliminary results quickly.

Ranked in increasing lengths, the 20 shortest English articles are as follows:

In [1]:
shortest_articles = ['Polar_bear', 
                     'Mount_Rushmore',
                     'Potato',
                     'Burj_Khalifa',
                     'Machu_Picchu',
                     'Petra',
                     'Animal',
                     'Great_Wall_of_China',
                     'Angkor_Wat',
                     'Taj_Mahal',
                     'Colosseum',
                     'Japan',
                     'Elizabeth_II',
                     'Apple',
                     'Photosynthesis',
                     'Coronavirus',
                     'Lionel_Messi',
                     'Eiffel_Tower',
                     'Large_Hadron_Collider',
                     'Monaco']

#### EasyOCR

[EasyOCR](https://github.com/JaidedAI/EasyOCR) relies on a text detection model using the CRAFT algorithm and a recognition model using a Convolutional Recurrent Neural Network (CRNN). EasyOCR supports over 80 languages, including the ones I wish to test.

Refer to EasyOCR's [supported languages](https://www.jaided.ai/easyocr/) for the mappings of language to EasyOCR language codes. 

In [1]:
import easyocr

# This needs to run only once to load the model into memory
en_reader = easyocr.Reader(['en']) 
th_reader = easyocr.Reader(['th'])
vi_reader = easyocr.Reader(['vi'])
id_reader = easyocr.Reader(['id'])

In [27]:
from ocr import run_easy_ocr, run_easy_ocr_on_all

# run_easy_ocr_on_all('../data/thai', th_reader)
# run_easy_ocr_on_all('../data/english', en_reader)
# run_easy_ocr_on_all('../data/vietnamese', vi_reader)
# run_easy_ocr_on_all('../data/bahasa', id_reader)

# run_easy_ocr(shortest_articles, '../data/thai', th_reader)
# run_easy_ocr(shortest_articles, '../data/english', en_reader)
# run_easy_ocr(shortest_articles, '../data/vietnamese', vi_reader)
# run_easy_ocr(shortest_articles, '../data/bahasa', id_reader)

Running on article: Polar_bear
Finished! Time taken: 10.072195053100586 seconds
Running on article: Mount_Rushmore
Finished! Time taken: 6.677720785140991 seconds
Running on article: Potato
Finished! Time taken: 13.164318084716797 seconds
Running on article: Burj_Khalifa
Finished! Time taken: 7.71886420249939 seconds
Running on article: Machu_Picchu
Finished! Time taken: 7.411307096481323 seconds
Running on article: Petra
Finished! Time taken: 41.22079014778137 seconds
Running on article: Animal
Finished! Time taken: 77.12726211547852 seconds
Running on article: Great_Wall_of_China
Finished! Time taken: 24.797030925750732 seconds
Running on article: Angkor_Wat
Finished! Time taken: 24.021788120269775 seconds
Running on article: Taj_Mahal
Finished! Time taken: 4.959687948226929 seconds
Running on article: Colosseum
Finished! Time taken: 13.12365174293518 seconds
Running on article: Japan
Finished! Time taken: 81.3571708202362 seconds
Running on article: Elizabeth_II
Finished! Time taken

#### Tesseract

[Tesseract](https://github.com/tesseract-ocr/tesseract) uses a Long Short-Term Memory (LSTM) Recurrent Neural Network (RNN) architecture.

Running the `pytesseract` library requires installing Tesseract. On macOS, I recommend using Homebrew:

```bash
brew install tesseract 

# To enable support for extra languages
brew install tesseract-lang 
```

In [32]:
from ocr import run_tesseract, run_tesseract_on_all

# run_tesseract_on_all('../data/thai', 'tha')
# run_tesseract_on_all('../data/english', 'eng')
# run_tesseract_on_all('../data/vietnamese', 'vie')
# run_tesseract_on_all('../data/bahasa', 'ind')

run_tesseract(shortest_articles, '../data/thai', 'tha')
run_tesseract(shortest_articles, '../data/english', 'eng')
run_tesseract(shortest_articles, '../data/vietnamese', 'vie')
run_tesseract(shortest_articles, '../data/bahasa', 'ind')

Running on article: Polar_bear
Finished! Time taken: 1.6898150444030762 seconds
Running on article: Mount_Rushmore
Finished! Time taken: 25.245693922042847 seconds
Running on article: Potato
Finished! Time taken: 31.095876932144165 seconds
Running on article: Burj_Khalifa
Finished! Time taken: 35.45262694358826 seconds
Running on article: Machu_Picchu
Finished! Time taken: 31.326441764831543 seconds
Running on article: Petra
Finished! Time taken: 34.52344608306885 seconds
Running on article: Animal
Finished! Time taken: 40.00341296195984 seconds
Running on article: Great_Wall_of_China
Finished! Time taken: 24.967286109924316 seconds
Running on article: Angkor_Wat
Finished! Time taken: 31.587785005569458 seconds
Running on article: Taj_Mahal
Finished! Time taken: 33.6811089515686 seconds
Running on article: Colosseum
Finished! Time taken: 30.798343896865845 seconds
Running on article: Japan
Finished! Time taken: 344.9951889514923 seconds
Running on article: Elizabeth_II
Finished! Time t

#### Preprocessing Results



#### Evaluation

To evaluate the accuracy of the OCR tools, I chose to use Character Error Rate (CER) and Word Error Rate (WER) using the [jiwer](https://jitsi.github.io/jiwer/) library. The values are rounded to 4 decimal places and exported into a CSV file.

I previously tried [torchmetrics](https://lightning.ai/docs/torchmetrics/stable/), but it took 10-20 minutes to evaluate results for 1 article. jiwer is able to evaluate results in milliseconds.


In [2]:
from evaluate import evaluate_to_csv

languages = ['english', 'thai', 'bahasa', 'vietnamese']

for language in languages:
    evaluate_to_csv(f'../data/{language}', 'text-clean.txt', 'easy-ocr-results-clean.txt', 'easy-ocr-evaluation')
    evaluate_to_csv(f'../data/{language}', 'text-clean.txt', 'tesseract-results-clean.txt', 'tesseract-evaluation')

### Iteration v2

